In [1]:
#!pip3 install pandas
#!pip3 install PyArrow
from pyspark.sql.functions import col

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

In [3]:
import os
import time
import json
import requests
import xml.etree.ElementTree as ET
import datetime
import subprocess

In [4]:
if os.path.exists("/etc/hadoop/conf/hive-site.xml"):
    tree = ET.parse("/etc/hadoop/conf/hive-site.xml")
    root = tree.getroot()
    for prop in root.findall("property"):
        if prop.find("name").text == "hive.metastore.warehouse.dir":
            storage = (
                prop.find("value").text.split("/")[0]
                + "//"
                + prop.find("value").text.split("/")[2]
            )

In [5]:
os.environ["STORAGE"] = storage

In [6]:
print(storage)

s3a://go01-demo


In [7]:
spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
    .config("spark.yarn.access.hadoopFileSystems",os.environ["STORAGE"])\
    .config("spark.rpc.message.maxSize", "1024")\
    .config("spark.dynamicAllocation.enabled", "true")\
    .getOrCreate()
 #   .config("spark.driver.cores", 4)\
 #   .config("spark.driver.memory", "8g")\
    

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


In [8]:
customers_df = spark.read.csv(os.environ["STORAGE"]+'/cde-workshop/clickthrough/customers/data', header=True)

In [9]:
customers_df.select("customer_id").show()

+-----------+
|customer_id|
+-----------+
|          1|
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
|         10|
|         11|
|         12|
|         13|
|         14|
|         15|
|         16|
|         17|
|         18|
|         19|
|         20|
+-----------+
only showing top 20 rows



In [10]:
!pip3 install faker

In [11]:
customers_df.dtypes

[('customer_id', 'string'),
 ('username', 'string'),
 ('name', 'string'),
 ('gender', 'string'),
 ('email', 'string'),
 ('occupation', 'string'),
 ('birthdate', 'string'),
 ('address', 'string')]

In [29]:
max_current_cust_id = int(customers_df.select(F.max("customer_id")).collect()[0]['max(customer_id)'])

In [23]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [24]:
from faker import Faker
fake = Faker(seed=3)

In [25]:
!pip3 install pandas
import pandas as pd

In [26]:
appends = []
for i in range(5):
    appends.append(fake.profile())

In [27]:
appends_df = pd.DataFrame(appends)[["username", "name", "sex", "mail", "job", "birthdate", "address"]]

In [38]:
import random
import numpy as np

In [28]:
def make_batch_df(max_current_cust_id):
    batch_size = random.randint(100, 1000)
    n
    appends = []
    for i in range(batch_size):
        appends.append(fake.profile())
    appends_df = pd.DataFrame(appends)[["username", "name", "sex", "mail", "job", "birthdate", "address"]]
    appends_df['customer_id'] =  

username     object
name         object
sex          object
mail         object
job          object
birthdate    object
address      object
dtype: object

In [61]:
customers_df = customers_df.drop(*["salary", "zip"])

In [70]:
customers_df.show()

22/10/25 02:29:48 232 ERROR TaskSetManager: Task 0 in stage 11.0 failed 4 times; aborting job


Py4JJavaError: An error occurred while calling o116.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 4 times, most recent failure: Lost task 0.3 in stage 11.0 (TID 25) (100.100.137.93 executor 5): java.io.FileNotFoundException: 
No such file or directory: s3a://go01-demo/cde-workshop/clickthrough/customer_data.csv

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:119)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:164)
	at org.apache.spark.sql

In [69]:
customers_df.write.mode("overwrite").csv(os.environ["STORAGE"]+'/cde-workshop/clickthrough/customers/data', header=True)

22/10/25 02:28:49 095 ERROR TaskSetManager: Task 0 in stage 10.0 failed 4 times; aborting job
22/10/25 02:28:49 097 ERROR FileFormatWriter: Aborting job 4354a334-ea2e-4ccf-a562-03f94531bb62.
org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 (TID 21) (100.100.137.93 executor 5): org.apache.hadoop.fs.FileAlreadyExistsException: s3a://go01-demo/cde-workshop/clickthrough/customers/data/part-00000-e0399843-ebe4-4a64-94f6-59681fd21826-c000.csv already exists
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerCreateFile(S3AFileSystem.java:1647)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$create$6(S3AFileSystem.java:1599)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.lambda$trackDurationOfOperation$5(IOStatisticsBinding.java:499)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.trackDuration(IOStatisticsBinding.java:444)
	at org.apache.hadoop.fs.s3a.S3AFileSystem

Py4JJavaError: An error occurred while calling o145.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:111)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at org.ap